In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from ibm_watsonx_ai import Credentials
from langchain_ibm import WatsonxLLM, WatsonxEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
import gradio as gr
import warnings, logging

warnings.filterwarnings("ignore")
logging.getLogger("posthog").setLevel(logging.ERROR)

# === Get LLM ===
def get_llm():
    model_id = "ibm/granite-3-2-8b-instruct"
    parameters = {
        GenParams.MAX_NEW_TOKENS: 512,
        GenParams.TEMPERATURE: 0.5,
    }
    watsonx_llm = WatsonxLLM(
        model_id=model_id,
        url="https://us-south.ml.cloud.ibm.com",
        project_id="skills-network",
        params=parameters,
    )
    return watsonx_llm

# === Document Loader ===
def document_loader(file):
    loader = PyPDFLoader(file.name)
    loaded_document = loader.load()
    return loaded_document

# === Text Splitter ===
def text_splitter(data):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    chunks = splitter.split_documents(data)
    chunks = [c for c in chunks if c.page_content.strip()]  # filter kosong
    return chunks

# === Embedding Model ===
def watsonx_embedding():
    embed_params = {
        EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
        EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
    }
    watsonx_embedding = WatsonxEmbeddings(
        model_id="ibm/slate-125m-english-rtrvr-v2",  # ✅ pakai v2
        url="https://us-south.ml.cloud.ibm.com",
        project_id="skills-network",
        params=embed_params,
    )
    return watsonx_embedding

# === Vector DB ===
def vector_database(chunks):
    embedding_model = watsonx_embedding()
    vectordb = Chroma.from_documents(chunks, embedding_model)
    return vectordb

# === Retriever ===
def retriever(file):
    docs = document_loader(file)
    chunks = text_splitter(docs)
    vectordb = vector_database(chunks)
    retriever = vectordb.as_retriever()
    return retriever

# === QA Chain ===
def retriever_qa(file, query):
    llm = get_llm()
    retriever_obj = retriever(file)
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever_obj,
        return_source_documents=False
    )
    response = qa.invoke(query)  # ✅ gunakan query user
    return response["result"]

# === Gradio App ===
rag_application = gr.Interface(
    fn=retriever_qa,
    allow_flagging="never",
    inputs=[
        gr.File(label="📄 Upload PDF File", file_count="single", file_types=[".pdf"]),
        gr.Textbox(label="💬 Ask your question", lines=2, placeholder="Type your question here..."),
    ],
    outputs=gr.Textbox(label="🤖 AI Answer"),
    title="📘 WatsonX PDF Chatbot (RAG)",
    description="Upload a PDF document and ask any question — the chatbot will answer using the document content.",
)

rag_application.launch(server_name="127.0.0.1", server_port=7860)
